In [1]:
#from app import calculatePath
from datetime import datetime, timedelta
from src.SPICE.ephemeris import ephemeris
from src.traj import trajectory
import spiceypy as sp
import numpy as np
import os

ephem = ephemeris(sp, 'C:/Users/HP/Desktop/GravAssist')
earth = ephem.get_body('EARTH')
mars = ephem.get_body('MARS')
jupiter = ephem.get_body('JUPITER')
neptune = ephem.get_body('NEPTUNE')
sun = ephem.get_body('SUN')
venus = ephem.get_body('VENUS')
#res = sorted_transfers(earth, mars, datetime.now(), datetime.now()+timedelta(days=365), ephem.get_body('SUN').Gmass[0])

<IPython.core.display.Javascript object>

In [11]:
res = calculatePath(earth, mars, datetime.now(), datetime.now()+timedelta(days=365*100), ephem.get_body('SUN'))

In [15]:
res[0].deltaV

27.995839604840178

In [55]:
from src.lambert import *
from datetime import datetime, timedelta
from src.optimize import *



def direct_transfer_cost(body1, body2, t0, T, GM):
    s1 = body1.state(t0)
    s2 = body2.state(t0+T)
    p1, v1 = s1[:3], s1[3:]
    p2, v2 = s2[:3], s2[3:]
    sols = lambert_transfer(p1, p2, T, GM, prec=1e2)
    dVmin, res = None, None
    for sol in sols:
        v0, vf = sol['v1'], sol['v2']
        dV = np.linalg.norm(v0-v1) + np.linalg.norm(vf-v2)
        if not dVmin or dV < dVmin: dVmin, res = dV, sol
    return dVmin, res

earliest, latest = datetime.now(), datetime.now() + timedelta(days=300)

GM = ephem.get_body('SUN').const('GM', 1)[0]
F = lambda t0, T: direct_transfer(earth, mars, t0, T, GM)[0]
C = lambda t0, T: t0 + T < latest

import time
import random
start = time.perf_counter()

k = 5
inits = []
for i in range(k):
    for j in range(k):
        t0_ = earliest + (latest-earliest) * i / k
        T_ = (latest - t0_) * j / k
        dV = F(t0_, T_)
        if dV: inits.append([dV, t0_, T_])
inits = sorted(inits, key=lambda x: x[0])

end = time.perf_counter()
print(end - start)

ranges = [[earliest, latest], [timedelta(days=0), latest-earliest]]
steps = [(latest-earliest)/k/k, (latest-earliest)/k/k]        

N = 1
solutions = []
for initial in inits[:N]:
    pinit = initial[1:]
    res = decaying_descent(F, ranges, steps, pinit,
               condition=C, iters = 2, decay_factor=k)
    dV, solution = direct_transfer(earth, mars, res[0], res[1], GM)
    solutions.append([dV, solution])

solutions = sorted(solutions, key=lambda x: x[0])
    
end = time.perf_counter()
print(end - start)

#direct_transfer(earth, mars, )


8.672824599999785
17.630401899999924


In [56]:
solutions

[[6.62829370457847,
  {'v1': array([28.52242473, 14.71318485,  7.69728694]),
   'v2': array([-21.03811167,   3.84464121,   1.02353396]),
   'dt': datetime.timedelta(days=203, seconds=86389, microseconds=453557)}]]

In [11]:
from app import LambertCalculator
a = LambertCalculator(mars, earth, datetime.now(), datetime.now()+timedelta(days=400), sun)
res = a.run()

In [12]:
res[0].deltaV

6.3232398661008435

In [ ]:
from app import MissionCalculator
from src.SPICE.ephemeris import ephemeris
import spiceypy as sp



global ephem
ephem = ephemeris(sp, 'C:/Users/HP/Desktop/GravAssist')

m = MissionCalculator('VOYAGER 1')

res = m.run(ephem)

<IPython.core.display.Javascript object>

In [2]:
res

[<src.path.path at 0x1bc19042cc8>, <src.path.path at 0x1bc18d22848>]